#### Import Library

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from matplotlib import font_manager

font_fname = 'C:/Windows/Fonts/malgun.ttf'
font_family = font_manager.FontProperties(fname=font_fname).get_name()
plt.rcParams["font.family"] = font_family

#### 엑세스 토큰 발급

In [23]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote

id = '0abacdbf9fd946b3a13b'
key = 'ca78b8da40044148978c'
url = 'https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json'

query = '?' + urlencode(
{
    quote_plus('consumer_key'):id,
    quote_plus('consumer_secret'):key
})

response = requests.get(url+query)
acc_token = response.json()['result']['accessToken']

#### 전국, 시군구 단위 인구 총조사지표

In [34]:
url = 'https://sgisapi.kostat.go.kr/OpenAPI3/stats/population.json'

query = '?' + urlencode(
{
    quote_plus('accessToken'):acc_token,
    quote_plus('year'):2020,
    quote_plus('low_search'):1
})

response = requests.get(url+query)
total_sido = response.json()['result']
total_sido

[{'imga_ppltn': '3016',
  'adm_cd': '11',
  'tot_house': '2918742',
  'ppltn_dnsty': '15090.4',
  'tot_ppltn': '9134216',
  'aged_child_idx': '153.9',
  'corp_cnt': 'N/A',
  'imga_cnt': '1169',
  'employee_cnt': 'N/A',
  'haesuoga_cnt': '14',
  'juv_suprt_per': '14.2',
  'haesuoga_ppltn': '27',
  'tot_family': '3982290',
  'adm_nm': '서울특별시',
  'nongga_cnt': '8418',
  'naesuoga_ppltn': '52',
  'avg_age': '42.6',
  'avg_fmember_cnt': '2.3',
  'nongga_ppltn': '21897',
  'naesuoga_cnt': '18',
  'oldage_suprt_per': '21.9'},
 {'imga_ppltn': '3025',
  'adm_cd': '21',
  'tot_house': '1162449',
  'ppltn_dnsty': '4160.6',
  'tot_ppltn': '3264419',
  'aged_child_idx': '179.3',
  'corp_cnt': 'N/A',
  'imga_cnt': '1227',
  'employee_cnt': 'N/A',
  'haesuoga_cnt': '1816',
  'juv_suprt_per': '15.6',
  'haesuoga_ppltn': '4266',
  'tot_family': '1405037',
  'adm_nm': '부산광역시',
  'nongga_cnt': '11967',
  'naesuoga_ppltn': '347',
  'avg_age': '44.6',
  'avg_fmember_cnt': '2.3',
  'nongga_ppltn': '28529',


In [52]:
total_sido = pd.DataFrame(total_sido)[['adm_cd','adm_nm','tot_ppltn','ppltn_dnsty','aged_child_idx','oldage_suprt_per']]
total_sido

,adm_cd,adm_nm,tot_ppltn,ppltn_dnsty,aged_child_idx,oldage_suprt_per
0,11,서울특별시,9134216,15090.4,153.9,21.9
1,21,부산광역시,3264419,4160.6,179.3,28
2,22,대구광역시,2350349,2672.1,139.7,23.5
3,23,인천광역시,2816119,2552.3,112.1,19.4
4,24,광주광역시,1431652,2874.8,106.9,20
5,25,대전광역시,1451305,2690.2,114,19.6
6,26,울산광역시,1097515,1033.6,91.1,17.4
7,29,세종특별자치시,344203,740.5,47,13.5
8,31,경기도,12802161,1243.2,97.8,18.5
9,32,강원도,1478378,87.9,183.2,30.7


In [53]:
codes = [11, 21, 22, 23, 24, 25, 26, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39]
regs = ['seoul', 'busan', 'daegu', 'incheon', 'gwangju', 'daejun', 'woolsan', 'sejong', 'gyunggi', 'gangwon', 'choongbuk', 'choongnam',
      'junbuk', 'junnam', 'gyeongbuk', 'gyeongnam', 'jeju']

url = 'https://sgisapi.kostat.go.kr/OpenAPI3/stats/population.json'
total_sigungu = pd.DataFrame()

for code, reg in zip(codes, regs):
    query = '?' + urlencode(
    {
        quote_plus('accessToken'):acc_token,
        quote_plus('year'):2020,
        quote_plus('adm_cd'):code,
        quote_plus('low_search'):1
    })

    response = requests.get(url+query)
    globals()[f'{reg}_sido'] = response.json()['result']
    temp = pd.DataFrame(response.json()['result'])[['adm_cd','adm_nm','tot_ppltn','ppltn_dnsty','aged_child_idx','oldage_suprt_per']]
    total_sigungu = pd.concat([total_sigungu, temp])

In [54]:
total_sigungu

,adm_cd,adm_nm,tot_ppltn,ppltn_dnsty,aged_child_idx,oldage_suprt_per
0,11010,종로구,138229,5766.1,210.9,24.4
1,11020,중구,116112,11631.3,226,24.6
2,11030,용산구,206249,9428,179,22.2
3,11040,성동구,278665,16582.9,150.1,20.6
4,11050,광진구,330853,19420.9,156.5,18.9
...,...,...,...,...,...,...
19,38380,함양군,36264,50.3,400.3,60.8
20,38390,거창군,58310,73,264.1,46.2
21,38400,합천군,41107,41.8,626.6,75.6
0,39010,제주시,469462,477.2,96,20.8


#### 전국, 시군구 단위 인구통계

In [44]:
# 전국고령인구
url = 'https://sgisapi.kostat.go.kr/OpenAPI3/stats/searchpopulation.json'

query = '?' + urlencode(
{
    quote_plus('accessToken'):acc_token,
    quote_plus('year'):2020,
    quote_plus('age_type'):24,
})

response = requests.get(url+query)
total_pop = response.json()['result']
total_pop

[{'adm_cd': '11',
  'adm_nm': '서울특별시',
  'avg_age': '84.6',
  'population': '1470127'},
 {'adm_cd': '21',
  'adm_nm': '부산광역시',
  'avg_age': '83.7',
  'population': '636709'},
 {'adm_cd': '22',
  'adm_nm': '대구광역시',
  'avg_age': '83.9',
  'population': '393032'},
 {'adm_cd': '23',
  'adm_nm': '인천광역시',
  'avg_age': '83.9',
  'population': '399673'},
 {'adm_cd': '24',
  'adm_nm': '광주광역시',
  'avg_age': '84.3',
  'population': '206700'},
 {'adm_cd': '25',
  'adm_nm': '대전광역시',
  'avg_age': '84.2',
  'population': '208173'},
 {'adm_cd': '26', 'adm_nm': '울산광역시', 'avg_age': '83', 'population': '139620'},
 {'adm_cd': '29',
  'adm_nm': '세종특별자치시',
  'avg_age': '84.1',
  'population': '32753'},
 {'adm_cd': '31', 'adm_nm': '경기도', 'avg_age': '84.3', 'population': '1724276'},
 {'adm_cd': '32', 'adm_nm': '강원도', 'avg_age': '83.3', 'population': '307691'},
 {'adm_cd': '33', 'adm_nm': '충청북도', 'avg_age': '83.6', 'population': '278984'},
 {'adm_cd': '34', 'adm_nm': '충청남도', 'avg_age': '83.7', 'population': '3

In [55]:
pop_sido = pd.DataFrame(total_pop)
pop_sido

,adm_cd,adm_nm,avg_age,population
0,11,서울특별시,84.6,1470127
1,21,부산광역시,83.7,636709
2,22,대구광역시,83.9,393032
3,23,인천광역시,83.9,399673
4,24,광주광역시,84.3,206700
5,25,대전광역시,84.2,208173
6,26,울산광역시,83,139620
7,29,세종특별자치시,84.1,32753
8,31,경기도,84.3,1724276
9,32,강원도,83.3,307691


In [61]:
# 시군구 노령인구
codes = [11, 21, 22, 23, 24, 25, 26, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39]
regs = ['seoul', 'busan', 'daegu', 'incheon', 'gwangju', 'daejun', 'woolsan', 'sejong', 'gyunggi', 'gangwon', 'choongbuk', 'choongnam',
      'junbuk', 'junnam', 'gyeongbuk', 'gyeongnam', 'jeju']

url = 'https://sgisapi.kostat.go.kr/OpenAPI3/stats/searchpopulation.json'
pop_sigungu = pd.DataFrame()

for code, reg in zip(codes, regs):
    query = '?' + urlencode(
    {
        quote_plus('accessToken'):acc_token,
        quote_plus('year'):2020,
        quote_plus('adm_cd'):code,
        quote_plus('age_type'):24,
        quote_plus('low_search'):1
    })

    response = requests.get(url+query)
    globals()[f'{reg}_pop'] = response.json()['result']
    temp = pd.DataFrame(response.json()['result'])
    pop_sigungu = pd.concat([pop_sigungu, temp])

In [62]:
pop_sigungu

,adm_cd,adm_nm,avg_age,population
0,11010,종로구,84.1,24819
1,11020,중구,83.6,21075
2,11030,용산구,84.9,34071
3,11040,성동구,84.1,42710
4,11050,광진구,84.1,47752
...,...,...,...,...
19,38380,함양군,83.8,12533
20,38390,거창군,84.1,16451
21,38400,합천군,83.1,16556
0,39010,제주시,85.1,68532


In [58]:
# 60~64세 (향후 5년 내 노인인구 편입)
url = 'https://sgisapi.kostat.go.kr/OpenAPI3/stats/searchpopulation.json'

query = '?' + urlencode(
{
    quote_plus('accessToken'):acc_token,
    quote_plus('year'):2020,
    quote_plus('age_type'):13,
})

response = requests.get(url+query)
elder_sido = pd.DataFrame(response.json()['result'])
elder_sido

,adm_cd,adm_nm,avg_age,population
0,11,서울특별시,62,679203
1,21,부산광역시,62,289201
2,22,대구광역시,62,183730
3,23,인천광역시,62,216383
4,24,광주광역시,62,97571
5,25,대전광역시,62,103607
6,26,울산광역시,62,85330
7,29,세종특별자치시,62,17086
8,31,경기도,62,885995
9,32,강원도,62,134913


In [63]:
codes = [11, 21, 22, 23, 24, 25, 26, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39]
regs = ['seoul', 'busan', 'daegu', 'incheon', 'gwangju', 'daejun', 'woolsan', 'sejong', 'gyunggi', 'gangwon', 'choongbuk', 'choongnam',
      'junbuk', 'junnam', 'gyeongbuk', 'gyeongnam', 'jeju']

url = 'https://sgisapi.kostat.go.kr/OpenAPI3/stats/searchpopulation.json'
elder_sigungu = pd.DataFrame()

for code, reg in zip(codes, regs):
    query = '?' + urlencode(
    {
        quote_plus('accessToken'):acc_token,
        quote_plus('year'):2020,
        quote_plus('adm_cd'):code,
        quote_plus('age_type'):13,
        quote_plus('low_search'):1
    })

    response = requests.get(url+query)
    globals()[f'soon_{reg}_pop'] = response.json()['result']
    temp = pd.DataFrame(response.json()['result'])
    elder_sigungu = pd.concat([elder_sigungu, temp])
elder_sigungu

,adm_cd,adm_nm,avg_age,population
0,11010,종로구,62,10364
1,11020,중구,62,9182
2,11030,용산구,62,15109
3,11040,성동구,62,20444
4,11050,광진구,62,23755
...,...,...,...,...
19,38380,함양군,62,3828
20,38390,거창군,62,5454
21,38400,합천군,62,4695
0,39010,제주시,62,31314


#### Folium map

In [127]:
# 행정구역경계

url = 'https://sgisapi.kostat.go.kr/OpenAPI3/boundary/hadmarea.geojson'

query = '?' + urlencode(
{
    quote_plus('accessToken'):acc_token,
    quote_plus('year'):2021
})

response = requests.get(url+query)
geo_path = response.json()
geo_path

{'type': 'FeatureCollection',
 'id': 'API_0704',
 'errMsg': 'Success',
 'errCd': 0,
 'trId': 'P5D6_API_0704_1652137678858',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[957875.2813999983, 1966791.6757],
      [958154.3994999977, 1966774.6934999975],
      [958198.4332999984, 1966797.8854999994],
      [958294.2949999976, 1966848.3881000069],
      [958295.4335999993, 1966857.9390999977],
      [958305.7208000013, 1966944.221100002],
      [958364.2567, 1966924.1739000035],
      [958478.4707999995, 1966781.6821000031],
      [958504.2289999977, 1966740.8723000006],
      [958526.6136999984, 1966638.8986000007],
      [958531.4497999989, 1966616.8677000008],
      [958535.9174, 1966575.953099999],
      [958536.1887999987, 1966567.9134000023],
      [958541.6651999971, 1966405.7050999994],
      [958654.215799999, 1966050.3329000075],
      [958775.9154999993, 1965912.6473000026],
      [958880.2520999995, 1965932.3607999994],
      [959059.

In [129]:
geo_path['features']

17

In [107]:
temp_geo = total_sido.set_index('adm_nm')
temp_geo

,adm_cd,tot_ppltn,ppltn_dnsty,aged_child_idx,oldage_suprt_per
adm_nm,,,,,
서울특별시,11,9134216,15090.4,153.9,21.9
부산광역시,21,3264419,4160.6,179.3,28
대구광역시,22,2350349,2672.1,139.7,23.5
인천광역시,23,2816119,2552.3,112.1,19.4
광주광역시,24,1431652,2874.8,106.9,20
대전광역시,25,1451305,2690.2,114,19.6
울산광역시,26,1097515,1033.6,91.1,17.4
세종특별자치시,29,344203,740.5,47,13.5
경기도,31,12802161,1243.2,97.8,18.5


In [131]:
import folium
map = folium.Map(location=[37.5502, 126.982], zoom_start=11)

folium.Choropleth(geo_data = geo_path,
               data = temp_geo['tot_ppltn'],
               columns = [temp_geo.index,'tot_ppltn'],
               fill_color = 'PuRd',
               key_on = 'features.properties.adm_nm').add_to(map)

map

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [79]:
geo_path['features'][0]['properties']

{'adm_cd': '11', 'adm_nm': '서울특별시', 'y': '1952053', 'x': '953932'}

In [75]:
temp_geo.index

Index(['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시',
       '세종특별자치시', '경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도', '경상남도',
       '제주특별자치도'],
      dtype='object', name='adm_nm')